In [1]:
#importando bibliotecas

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('titanic_dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.shape

(891, 12)

Ao fazer a submissão no kaggle com todos os gêneros "female" = 1, e "male" = 0. Obteve-se um score de 0,76
Primeira meta a ser batida

In [10]:
#Analisando coluna Sex

df['Sex'].unique()

array(['male', 'female'], dtype=object)

In [11]:
df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [12]:
#Transformando valores em 1 e 0 - 1 female / 0 male

def sex_1_0(sex):
    if sex == 'female':
        return 1
    else:
        return 0

df['Sex_1_0'] = df['Sex'].map(sex_1_0)

In [13]:
df.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_1_0
627,628,1,1,"Longley, Miss. Gretchen Fiske",female,21.0,0,0,13502,77.9583,D9,S,1
364,365,0,3,"O'Brien, Mr. Thomas",male,NaN,1,0,370365,15.5000,NaN,Q,0
867,868,0,1,"Roebling, Mr. Washington Augustus II",male,31.0,0,0,PC 17590,50.4958,A24,S,0


In [14]:
#primeiro modelo - verificar se a voluna idade ('Age'), melhorará a resposta do modelo

#Verificando valores de idade. procurando por idades negativas/nulas ou valores extremamente altos.
df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [15]:
df['Age'].isnull().values.any() #valores nulos em idade

True

In [16]:
df['Age'].isna().sum()

177

In [17]:
df.fillna(-1, inplace=True)

In [19]:
variaveis = ['Sex_1_0', 'Age']

In [21]:
X = df[variaveis]
y = df['Survived']

In [22]:
#dividindo os dados

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [24]:
#Criando o modelo
from sklearn.ensemble import RandomForestClassifier
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)

In [26]:
modelo.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=0)

In [27]:
p = modelo.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, p)

0.7511210762331838

A inserção da coluna 'Age' não acrescentou poder de predição ao modelo. Fazendo validação cruzada para atestar.

In [43]:
from sklearn.model_selection import RepeatedKFold

resultados = []

kf = RepeatedKFold(n_splits = 2, n_repeats=10, random_state=10)

for linhas_treino, linhas_valid in kf.split(X):
        X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
        y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid]

        modelo = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=0)
        modelo.fit(X_treino, y_treino)

        p = modelo.predict(X_valid)

        acc = np.mean(y_valid == p)
        resultados.append(acc)

        print("Acc:", acc)

Acc: 0.7511210762331838
Acc: 0.7617977528089888
Acc: 0.7533632286995515
Acc: 0.7595505617977528
Acc: 0.7690582959641256
Acc: 0.7573033707865169
Acc: 0.7399103139013453
Acc: 0.7595505617977528
Acc: 0.7354260089686099
Acc: 0.797752808988764
Acc: 0.7690582959641256
Acc: 0.7640449438202247
Acc: 0.7757847533632287
Acc: 0.7573033707865169
Acc: 0.7556053811659192
Acc: 0.7640449438202247
Acc: 0.7443946188340808
Acc: 0.7595505617977528
Acc: 0.7533632286995515
Acc: 0.7640449438202247


In [32]:
np.mean(resultados)

0.759601451100922

"A coluna Age não acrescentou nenhum valor ao dado."

In [33]:
#inserindo novas variavies

novas_variaveis = ['Pclass', 'SibSp', 'Parch', 'Fare']

In [37]:
#fazendo escala entre as variavies

from sklearn.preprocessing import scale

for col in novas_variaveis:
    df[col] = scale(df[col])

In [47]:
X = df[variaveis + novas_variaveis].fillna(-1)
y = df['Survived']

In [48]:
from sklearn.model_selection import RepeatedKFold

resultados = []

kf = RepeatedKFold(n_splits = 2, n_repeats=10, random_state=10)

for linhas_treino, linhas_valid in kf.split(X):
        X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
        y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid]

        modelo = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=0)
        modelo.fit(X_treino, y_treino)

        p = modelo.predict(X_valid)

        acc = np.mean(y_valid == p)
        resultados.append(acc)

        print("Acc:", acc)

Acc: 0.7892376681614349
Acc: 0.7797752808988764
Acc: 0.8295964125560538
Acc: 0.8179775280898877
Acc: 0.7847533632286996
Acc: 0.7842696629213484
Acc: 0.8161434977578476
Acc: 0.7820224719101123
Acc: 0.8004484304932735
Acc: 0.8044943820224719
Acc: 0.8228699551569507
Acc: 0.802247191011236
Acc: 0.8094170403587444
Acc: 0.8067415730337079
Acc: 0.8183856502242153
Acc: 0.7910112359550562
Acc: 0.8385650224215246
Acc: 0.8044943820224719
Acc: 0.7982062780269058
Acc: 0.8157303370786517


In [49]:
np.mean(resultados)

0.8048193681664735

Com as novas variaveis foi possível melhorar o modelo